# Automated Extraction of 2D Seed Phenotype Data using a pre-trained `StarDist` model, `Scivision`, and `Pixelflow`

### This notebook demonstrates how to use pixelflow to extract measurements of segmented objects from 2D imagery. This demo will use 2D lightbox images of Oilseed rape (*Brassica napus*) seedpods detected and segmented using an adapted StarDist model as described in [Corcoran et al. 2023](https://www.frontiersin.org/articles/10.3389/fpls.2023.1120182/full).

### Example images required to run this notebook can be found [here](https://zenodo.org/records/8355920)

### R code used to run valve sorting in this notebook is available from the [Scivision gallery github page](https://github.com/scivision-gallery/pixelflow_seed_demo/tree/main) seed (`'seedpod_2D_valve_lowess_single.R'`)

## 1. Load dependencies including `Scivision` and `Pixelflow`

In [1]:
from pixelflow import pixelflow

import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
from pathlib import Path

from glob import glob
import re

from scivision import default_catalog, load_pretrained_model

## 2. Find `StarDist Seed` model in `Scivision` model catalog

In [2]:
# load full scivision model catalog
models_catalog = default_catalog.models.to_dataframe()
models_catalog

name  \
0                         stardist   
1                          PlantCV   
2                      greenotyper   
3                  mapreader-plant   
4                resnet50-plankton   
5                   vedge-detector   
6                image-classifiers   
7          huggingface-classifiers   
8         huggingface-segmentation   
9     huggingface-object-detection   
10             detectreeRGB-forest   
11     flower-classification-model   
12  butterfly-classification-model   
13                    Affinity-VAE   
14                          btrack   
15                   StarDist Seed   

                                          description  \
0   Single class object detection and segementatio...   
1   Open-source image analysis software package ta...   
2   Greenotyper is an image analysis tool for larg...   
3   automated detection of plant patches in images...   
4   automated classification of plankton images co...   
5   automated detection of coastal vegetation edge...   
6   Scivision plugin for the image-classifiers pac...   
7   Selection of image classifiers taken from http...   
8   Selection of segmentation algorithms taken fro...   
9   Selection of object detection algorithms taken...   
10  Detect changes in tall tree height with machin...   
11                            Identify flower species   
12                         Identify butterfly species   
13  A framework for automatic clustering and class...   
14  btrack is a Python library for multi object tr...   
15  Pre-trained StarDist model for detection and s...   

                                                tasks  \
0   (TaskEnum.object_detection, TaskEnum.segmentat...   
1   (TaskEnum.object_detection, TaskEnum.segmentat...   
2                             (TaskEnum.segmentation)   
3   (TaskEnum.object_detection, TaskEnum.classific...   
4                          (TaskEnum.classificiation)   
5                             (TaskEnum.segmentation)   
6                          (TaskEnum.classificiation)   
7                          (TaskEnum.classificiation)   
8                             (TaskEnum.segmentation)   
9                         (TaskEnum.object_detection)   
10  (TaskEnum.object_detection, TaskEnum.segmentat...   
11                         (TaskEnum.classificiation)   
12                         (TaskEnum.classificiation)   
13                         (TaskEnum.classificiation)   
14                                                 ()   
15  (TaskEnum.object_detection, TaskEnum.segmentat...   

                                              pkg_url  \
0   git+https://github.com/stardist/stardist.git@m...   
1   git+https://github.com/danforthcenter/plantcv@...   
2   git+https://github.com/MarniTausen/Greenotyper...   
3   git+https://github.com/alan-turing-institute/m...   
4   git+https://github.com/alan-turing-institute/p...   
5   git+https://github.com/MartinSJRogers/VEdge_De...   
6   git+https://github.com/alan-turing-institute/s...   
7   git+https://github.com/alan-turing-institute/s...   
8   git+https://github.com/alan-turing-institute/s...   
9   git+https://github.com/alan-turing-institute/s...   
10  git+https://github.com/shmh40/detectreeRGB-tre...   
11  git+https://github.com/alan-turing-institute/f...   
12  git+https://github.com/alan-turing-institute/b...   
13  https://github.com/alan-turing-institute/affin...   
14                                             btrack   
15                                  stardist_seed_pod   

                                                  url  scivision_usable  \
0                https://github.com/stardist/stardist             False   
1           https://github.com/danforthcenter/plantcv             False   
2          https://github.com/MarniTausen/Greenotyper             False   
3   https://github.com/alan-turing-institute/mapre...              True   
4   https://github.com/alan-turing-institute/plank...              True   
5   https://github.com/

In [3]:
# get github repository link for the model we want to use (StarDist Seed)
model_repo = models_catalog[models_catalog.name == "StarDist Seed"].url.item()
model_repo

FlexibleUrl('https://raw.githubusercontent.com/alan-turing-institute/stardist-seed-pod/main/.scivision/model.yml', scheme='https', host='raw.githubusercontent.com', tld='com', host_type='domain', path='/alan-turing-institute/stardist-seed-pod/main/.scivision/model.yml')

## 3. Load `StarDist Seed` model from `Scivision` model catalog

In [4]:
# load pretrained seed model
model = load_pretrained_model(model_repo, model_selection= "model_2d", allow_install=True)
model

[11:59:50] INFO     Call='load_pretrained_model', module='scivision.io.reader', signature=(path:        ]8;id=735983;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/scivision/koala.py\koala.py]8;;\:]8;id=267354;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/scivision/koala.py#49\49]8;;\
                    os.PathLike, branch: str = 'main', allow_install: bool = False, model_selection:               
                    str = 'default', load_multiple: bool = False, *args, **kwargs) ->                              
                    scivision.io.wrapper.PretrainedModel, kwargs_given='model_selection, allow_install'            

2024-02-05 11:59:50.685966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 11:59:53.301841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.718646, nms_thresh=0.3.


scivision.PretrainedModel( 
  module='stardist_seed_pod', 
  model='model_2d', 
  source='https://github.com/alan-turing-institute/stardist-seed-pod' 
  pipe='DataPipe(input=<Parameter "X: numpy.ndarray">, output=<Parameter "image: numpy.ndarray">)' 
)

# Extract seed size and shape metrics from a single image (one seed pod) with `Pixelflow`

## 4. Load images

In [5]:
# load image
img = imread('01_Shengliyoucai_main_A.tif')
pod_name = '01_Shengliyoucai_main_A'

## 5. Predict location, size and shape of seeds with `StarDist Seed` model using `Scivision `core function

In [6]:
# detect and segment seeds in image
pred = model.predict(img)

In [7]:
# save seed labels created with Stardist Seed model
np.save(Path('content/' + pod_name + '_pred.npy'), pred)

## 6. Calculate seed size, shape and arrangement metrics with `Pixelflow`

In [8]:
# load image and mask
img = imread('01_Shengliyoucai_main_A.tif')
mask = pred

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(mask)

In [9]:
# create table of size and shape metrics
features = pixelflow(
    mask,
    img,
    spacing=(0.03,0.03),
    features=('label', 'bbox', 'centroid', 'area', 'equivalent_diameter', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'image_intensity')
)

YX image detected


In [10]:
# show table of size and shape metrics in millimetres
features

label  bbox-0  bbox-1  bbox-2  bbox-3  centroid-0  centroid-1     area  \
0       1     835    3742    1007    3946   27.659814  115.256195  26.5113   
1       2     846    3470    1014    3652   27.888554  106.812965  23.2209   
2       3     859    3185    1023    3350   28.205184   98.013696  20.5407   
3       4     861    4054    1022    4221   28.218190  124.127465  20.5902   
4       5     877    2725    1060    2948   29.050008   85.017869  30.9213   
5       6     881    4416    1030    4600   28.647920  135.176317  20.8566   
6       7     888    4241    1026    4417   28.731418  129.842832  18.3555   
7       8     907    4796    1064    4988   29.551040  146.720336  23.0472   
8       9     910    2463    1070    2615   29.664914   76.141714  18.5391   
9      10     924    4993    1093    5161   30.250303  152.281324  21.5325   
10     11     928    2178    1132    2370   30.876890   68.204226  29.6649   
11     12     928    5166    1124    5354   30.747954  157.770568  28.1178   
12     13    1003    5390    1160    5544   32.399653  163.984284  18.4554   
13     14    1014    1816    1163    2005   32.672187   57.230858  21.1680   
14     15    1026    1022    1198    1209   33.324746   33.445462  24.3837   
15     16    1035    3548    1234    3745   34.025832  109.394479  29.2050   
16     17    1049    4168    1243    4357   34.322070  127.856191  27.1971   
17     18    1053    3779    1246    3987   34.448106  116.472861  30.0879   
18     19    1057    3063    1208    3205   33.941341   94.005297  16.1892   
19     20    1059    2869    1220    3023   34.165903   88.331937  18.5319   
20     21    1085    4398    1258    4569   35.037302  134.469559  22.4838   
21     22    1097    2593    1271    2788   35.495790   80.648438  25.8336   
22     23    1099    4697    1309    4908   36.059014  144.046770  32.9904   
23     24    1126    4949    1327    5128   36.777866  151.144384  26.9064   
24     25    1130    1979    1319    2136   36.722909   61.707084  22.5612   
25     26    1176     549    1318     702   37.368127   18.757046  16.4781   
26     27    1176    1642    1358    1818   37.984530   51.900157  23.7411   
27     28    1210    1312    1386    1523   38.901697   42.474331  28.4454   
28     29    1250    5726    1398    5859   39.691402  173.777640  14.8824   

    equivalent_diameter  major_axis_length  minor_axis_length  orientation  \
0              5.809926           6.408291           5.293776    -1.531574   
1              5.437441           5.688197           5.217015     1.524893   
2              5.114023           5.160864           5.084435     1.232456   
3              5.120181           5.244345           5.017989    -1.373028   
4              6.274569           6.983425           5.664964    -1.470105   
5              5.153198           5.761219           4.633875    -1.469292   
6              4.834351           5.516655           4.257249    -1.566841   
7              5.417066           5.982115           4.933751    -1.492895   
8              4.858468           5.006396           4.731858    -0.154559   
9              5.236032           5.288282           5.204249     0.673634   
10             6.145773           6.356039           5.963699     0.027018   
11             5.983368           6.119983           5.870473    -0.122183   
12             4.847489           4.983515           4.733016    -0.726204   
13             5.191525           5.919740           4.578597    -1.524870   
14             5.571920           5.854066           5.324035    -1.540617   
15             6.097947           6.158044           6.053905    -0.286532   
16             5.884592           6.038060           5.751526    -0.487346   
17             6.189435           6.425920           5.974747    -1.522360   
18             4.540124           4.683783           4.416628    -0.189601   
19             4.857525           4.998235           4.734433     0.266099   
20             5.350445  

In [11]:
# output csv with measurements
features.to_csv('01_Shengliyoucai_main_A.csv', header=True, index=False)

## 7. Save segmented images of each individual seed with `Pixelflow`

In [12]:
# make list of all seeds in mask, shift backwards by one to match index values
X = list(features.features.label)
for i in range(len(X)):
    X[i] -= 1

In [13]:
# save segmented images of all seeds within a seed pod
for file in X:
    np.save(pod_name + '_seed_' + str(features.features.label[file]) + '.npy', features.image_intensity[file])

In [14]:
# load single seed image
seed_image = np.load('01_Shengliyoucai_main_A_seed_1.npy')

fig = plt.imshow(seed_image)

# Extract seed size and shape metrics and single seed images from multiple images (one seed pod per image) with `Pixelflow`

In [15]:
# load all input images
img_list = sorted(glob('content/*.tif'))
img_list

['content/DSC_0008.tif',
 'content/DSC_0017.tif',
 'content/DSC_0035.tif',
 'content/DSC_0039.tif']

In [16]:
for file in img_list:
  # load image and mask
  img = imread(file)
  pod_name = re.split("[/.]", file)[-2]
  pred = model.predict(img)
  np.save(Path('content/'+ pod_name + '_pred.npy'), pred)
  
  # create table of size and shape metrics
  features = pixelflow(
      mask=pred,
      image=img,
      spacing=(0.03,0.03),
      features=('label', 'bbox', 'centroid', 'area', 'equivalent_diameter', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'image_intensity'),
  )
  # output csv with measurements
  features.to_csv(pod_name + '.csv', header=True, index=False)

  # make list of all seeds in mask, shift backwards by one to match index values
  X = list(features.features.label)
  for i in range(len(X)):
      X[i] -= 1

  # save segmented images of all seeds within a seed pod
  for file in X:
      np.save(pod_name + '_seed_' + str(features.features.label[file]) + '.npy', features.image_intensity[file])

YX image detected
YX image detected
YX image detected
YX image detected


##  8. Sort detected seeds into 2 seeds pod valves, and calculated related measurements (order in valve from seed pod beak to pedicel, space between seeds in each valve) with `'seedpod_2D_valve_lowess_single.R'`

In [17]:
# import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import conversion, default_converter
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

In [18]:
# R packages to install
with conversion.localconverter(default_converter):
  utils = rpackages.importr('utils')
  utils.chooseCRANmirror(ind=1)
  packnames = ('gplots', 'tidyverse')
  # Selectively install what needs to be install.
  names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
  if len(names_to_install) > 0:
      utils.install_packages(StrVector(names_to_install))

In [19]:
# Defining the R script and loading the instance in Python
r = robjects.r

In [20]:
for file in img_list:
  # Calculate the image data in pixelflow
  img2 = imread(file)
  pod_name = re.split("[/.]", file)[-2]
  mask2 = np.load(Path('content/'+ pod_name + '_pred.npy'))
  features2 = pixelflow(mask2, img2, features=('label', 'bbox', 'centroid', 'area', 'equivalent_diameter', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'image_intensity'))

  # Loading the function we have defined in R.
  with conversion.localconverter(default_converter):
      r['source']('seedpod_2D_valve_lowess_single.R')
      seedpod_2D_single_r = robjects.globalenv['seedpod_2D_single']

  #converting dataframe into r object for passing into r function
  with (robjects.default_converter + pandas2ri.converter).context():
    r_from_pd_df = robjects.conversion.get_conversion().py2rpy(features2.features)

  #Invoking the R function and getting the result
  with conversion.localconverter(default_converter):
      df_result_r = seedpod_2D_single_r(r_from_pd_df, "plot_" + pod_name + ".png")

  #Converting it back to a pandas dataframe
  with (robjects.default_converter + pandas2ri.converter).context():
    df_result = robjects.conversion.get_conversion().rpy2py(df_result_r)


YX image detected
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[12:00:54] WARNING  R[write to console]:                                                           ]8;id=899543;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py\callbacks.py]8;;\:]8;id=481582;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py#124\124]8;;\
                    Attaching package: ‘gplots’                                                                    
                                                                                                                   
                                                                                                                   

           WARNING  R[write to console]: The following object is masked from ‘package:stats’:      ]8;id=305299;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py\callbacks.py]8;;\:]8;id=632937;file:///Users/ifenton/opt/anaconda3/envs/test_pxsd4/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py#124\124]8;;\
                                                                                                                   
                        lowess                                                                                     
                                                                                                                   
                                                                                                                   

YX image detected
YX image detected
YX image detected
